<a href="https://colab.research.google.com/github/kazirafi17/DataScrapping/blob/main/SerieA_last10years.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests


RAPIDAPI_KEY = 'b6e89817d6msh36107de73277139p116779jsne307fb015e33'

# Define the base URL for the fixtures endpoint
url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"

# List of seasons to fetch data for
seasons = ['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015']  # Season years

# Set the headers with your API key
headers = {
    "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
    "X-RapidAPI-Key": RAPIDAPI_KEY
}

all_fixtures = []  # To store fixtures for all seasons

# Loop through each season
for season in seasons:
    querystring = {
        "league": "135",  # Serie A league ID
        "season": season
    }

    # Make the API request for the current season
    response = requests.get(url, headers=headers, params=querystring)

    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Assuming the relevant data is in 'response' key
        fixtures = data['response']

        # Add the fixtures to the list
        all_fixtures.extend(fixtures)
    else:
        print(f"Error fetching season {season}: {response.status_code} - {response.text}")

# Convert to DataFrame (flattening any nested data)
df_fixtures = pd.json_normalize(all_fixtures)

# Save to CSV (optional)
df_fixtures.to_csv('Serie_A_full_stats_1.csv', index=False)
print("Data saved to Serie_A_full_stats_1.csv")

Data saved to Serie_A_full_stats_1.csv


In [3]:

RAPIDAPI_KEY = 'b6e89817d6msh36107de73277139p116779jsne307fb015e33'

# Define the base URLs for the fixtures and statistics endpoints
url_fixtures = "https://api-football-v1.p.rapidapi.com/v3/fixtures"
url_statistics = "https://api-football-v1.p.rapidapi.com/v3/fixtures/statistics"

headers = {
    "X-RapidAPI-Key": RAPIDAPI_KEY,
    "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

# Define league and season
league_id = "135"  # for Serie A
seasons = ['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015']

# Fetch the fixtures
querystring_fixtures = {"league": league_id, "season": seasons}
response_fixtures = requests.get(url_fixtures, headers=headers, params=querystring_fixtures)

# Check if the request was successful
if response_fixtures.status_code == 200:
    data_fixtures = response_fixtures.json()
    fixtures = data_fixtures['response']

    # Step 2: Loop through each fixture to get the full statistics
    all_stats = []
    for fixture in fixtures:
        fixture_id = fixture['fixture']['id']  # Get the fixture ID
        fixture_date = fixture['fixture']['date']  # Get the fixture date

        # Fetch the statistics for this fixture
        querystring_stats = {"fixture": fixture_id}
        response_stats = requests.get(url_statistics, headers=headers, params=querystring_stats)

        if response_stats.status_code == 200:
            stats_data = response_stats.json()

            # Extract relevant stats for both teams
            if 'response' in stats_data and stats_data['response']:
                for team_stats in stats_data['response']:
                    team_name = team_stats['team']['name']
                    team_id = team_stats['team']['id']

                    # Create a dictionary to store all statistics for this team in this fixture
                    stats_dict = {
                        "fixture_id": fixture_id,
                        "fixture_date": fixture_date,
                        "team_name": team_name,
                        "team_id": team_id
                    }

                    # Loop through the available statistics and store them in the dictionary
                    for stat in team_stats['statistics']:
                        stat_name = stat['type']
                        stat_value = stat['value']
                        stats_dict[stat_name] = stat_value

                    # Add the statistics to the list
                    all_stats.append(stats_dict)
        else:
            print(f"Error fetching stats for fixture {fixture_id}: {response_stats.status_code}")
else:
    print(f"Error fetching fixtures: {response_fixtures.status_code}")

# Step 3: Convert the statistics list to a DataFrame
df_stats = pd.DataFrame(all_stats)

# Save to CSV to retain all stats for each team per fixture
df_stats.to_csv('Serie_A_full_stats_2.csv', index=False)
print("Serie_A_full_stats_2.csv saved successfully.")


Serie_A_full_stats_2.csv saved successfully.


In [4]:
df1 = pd.read_csv('Serie_A_full_stats_1.csv')
df2 = pd.read_csv('Serie_A_full_stats_2.csv')

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3801 entries, 0 to 3800
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fixture.id              3801 non-null   int64  
 1   fixture.referee         3470 non-null   object 
 2   fixture.timezone        3801 non-null   object 
 3   fixture.date            3801 non-null   object 
 4   fixture.timestamp       3801 non-null   int64  
 5   fixture.periods.first   3468 non-null   float64
 6   fixture.periods.second  3468 non-null   float64
 7   fixture.venue.id        2699 non-null   float64
 8   fixture.venue.name      3801 non-null   object 
 9   fixture.venue.city      3800 non-null   object 
 10  fixture.status.long     3801 non-null   object 
 11  fixture.status.short    3801 non-null   object 
 12  fixture.status.elapsed  3468 non-null   float64
 13  league.id               3801 non-null   int64  
 14  league.name             3801 non-null   

In [9]:
# Rename the 'fixture.id' column to 'fixture_id'
df1 = df1.rename(columns={'fixture.id': 'fixture_id'})

In [10]:
final_df = pd.merge(df1, df2, on='fixture_id', how='inner')  # 'inner', 'outer', 'left', 'right'

In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 57 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fixture_id              760 non-null    int64  
 1   fixture.referee         760 non-null    object 
 2   fixture.timezone        760 non-null    object 
 3   fixture.date            760 non-null    object 
 4   fixture.timestamp       760 non-null    int64  
 5   fixture.periods.first   760 non-null    float64
 6   fixture.periods.second  760 non-null    float64
 7   fixture.venue.id        152 non-null    float64
 8   fixture.venue.name      760 non-null    object 
 9   fixture.venue.city      760 non-null    object 
 10  fixture.status.long     760 non-null    object 
 11  fixture.status.short    760 non-null    object 
 12  fixture.status.elapsed  760 non-null    float64
 13  league.id               760 non-null    int64  
 14  league.name             760 non-null    ob

In [12]:
final_df.to_csv('Serie_A_full_stats_last10years.csv', index=False)